# [Advent of Code 2019: Day 7](https://adventofcode.com/2019/day/7)

<h2>--- Day 7: Amplification Circuit ---</h2>
<p>Based on the navigational maps, you're going to need to send more power to your ship's thrusters to reach Santa in time. To do this, you'll need to configure a series of <a href="https://en.wikipedia.org/wiki/Amplifier">amplifiers</a> already installed
    on the ship.</p>
<p>There are five <span title="As you can see, I know exactly how rockets work.">amplifiers connected in series</span>; each one receives an input signal and produces an output signal. They are connected such that the first amplifier's output leads to the
    second amplifier's input, the second amplifier's output leads to the third amplifier's input, and so on. The first amplifier's input value is <code>0</code>, and the last amplifier's output leads to your ship's thrusters.</p>
<pre><code>    O-------O  O-------O  O-------O  O-------O  O-------O
0 -&gt;| Amp A |-&gt;| Amp B |-&gt;| Amp C |-&gt;| Amp D |-&gt;| Amp E |-&gt; (to thrusters)
    O-------O  O-------O  O-------O  O-------O  O-------O
</code></pre>
<p>The Elves have sent you some <em>Amplifier Controller Software</em> (your puzzle input), a program that should run on your <a href="https://adventofcode.com/2019/day/5">existing Intcode computer</a>. Each amplifier will need to run a copy of the program.</p>
<p>When a copy of the program starts running on an amplifier, it will first use an input instruction to ask the amplifier for its current <em>phase setting</em> (an integer from <code>0</code> to <code>4</code>). Each phase setting is used <em>exactly once</em>,
    but the Elves can't remember which amplifier needs which phase setting.</p>
<p>The program will then call another input instruction to get the amplifier's input signal, compute the correct output signal, and supply it back to the amplifier with an output instruction. (If the amplifier has not yet received an input signal, it waits
    until one arrives.)</p>
<p>Your job is to <em>find the largest output signal that can be sent to the thrusters</em> by trying every possible combination of phase settings on the amplifiers. Make sure that memory is not shared or reused between copies of the program.</p>
<p>For example, suppose you want to try the phase setting sequence <code>3,1,2,4,0</code>, which would mean setting amplifier <code>A</code> to phase setting <code>3</code>, amplifier <code>B</code> to setting <code>1</code>, <code>C</code> to <code>2</code>,
    <code>D</code> to <code>4</code>, and <code>E</code> to <code>0</code>. Then, you could determine the output signal that gets sent from amplifier <code>E</code> to the thrusters with the following steps:</p>
<ul>
    <li>Start the copy of the amplifier controller software that will run on amplifier <code>A</code>. At its first input instruction, provide it the amplifier's phase setting, <code>3</code>. At its second input instruction, provide it the input signal,
        <code>0</code>. After some calculations, it will use an output instruction to indicate the amplifier's output signal.</li>
    <li>Start the software for amplifier <code>B</code>. Provide it the phase setting (<code>1</code>) and then whatever output signal was produced from amplifier <code>A</code>. It will then produce a new output signal destined for amplifier <code>C</code>.</li>
    <li>Start the software for amplifier <code>C</code>, provide the phase setting (<code>2</code>) and the value from amplifier <code>B</code>, then collect its output signal.</li>
    <li>Run amplifier <code>D</code>'s software, provide the phase setting (<code>4</code>) and input value, and collect its output signal.</li>
    <li>Run amplifier <code>E</code>'s software, provide the phase setting (<code>0</code>) and input value, and collect its output signal.</li>
</ul>
<p>The final output signal from amplifier <code>E</code> would be sent to the thrusters. However, this phase setting sequence may not have been the best one; another sequence might have sent a higher signal to the thrusters.</p>
<p>Here are some example programs:</p>
<ul>
    <li>
        <p>Max thruster signal <em><code>43210</code></em> (from phase setting sequence <code>4,3,2,1,0</code>):</p>
        <pre><code>3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0</code></pre>
    </li>
    <li>
        <p>Max thruster signal <em><code>54321</code></em> (from phase setting sequence <code>0,1,2,3,4</code>):</p>
        <pre><code>3,23,3,24,1002,24,10,24,1002,23,-1,23,<br>101,5,23,23,1,24,23,23,4,23,99,0,0</code></pre>
    </li>
    <li>
        <p>Max thruster signal <em><code>65210</code></em> (from phase setting sequence <code>1,0,4,3,2</code>):</p>
        <pre><code>3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,<br>1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0</code></pre>
    </li>
</ul>
<p>Try every combination of phase settings on the amplifiers. <em>What is the highest signal that can be sent to the thrusters?</em></p>

In [1]:
import os
import unittest

class IntcodeComputer():
    
    OP_ADD = 1    
    OP_MULTIPLY = 2    
    OP_INPUT = 3    
    OP_OUTPUT = 4
    OP_JUMP_TRUE = 5
    OP_JUMP_FALSE = 6
    OP_LESS_THAN = 7
    OP_EQUALS = 8
    
    OP_HALT = 99
    
    PARAM_MODE_POS = '0'
    PARAM_MODE_IMD = '1'
    
    NOUN_ADDR = 1
    VERB_ADDR = 2
    RESULT_ADDR = 0
    START_ADDR = 0
    
    def __init__(self, data = []):
        self.inputs = []
        self.memory = []
        self.initial_memory = []
        if data:
            self.load_memory(data)
        
    def load_memory(self, data):
        self.initial_memory = self.normalize_memory(data)
        self.reset()
        
    def reset(self):
        if self.memory:
            del self.memory[:]
        self.memory = self.initial_memory.copy()
        
        if self.inputs:
            del self.inputs[:]
        self.inputs = []
        
        self.output = None
        self.last_input = None
        
        self.instruction_ptr = self.START_ADDR
        
    def add_input(self, data):
        self.inputs.append(data)
        
    def print_program(self):
        print("Program: {:02d}{:02d}".format(self.memory[self.NOUN_ADDR],self.memory[self.VERB_ADDR]))

    def normalize_memory(self, intcode):
        if type(intcode) is str:
            return list(map(int, intcode.split(',')))

        elif type(intcode) is list:
            if type(intcode[0]) is str:
                return list(map(int, intcode))
            else:
                return intcode

        else:
            raise Exception('Corrupt intcode')
            
    def get_paramater(self, mode):
        param = self.memory[self.instruction_ptr]
        self.instruction_ptr += 1
        
        if mode == self.PARAM_MODE_POS:
            val = self.memory[param]
        else:
            val = param

        return val
        
    def set_paramater(self, mode, data):
        param = self.memory[self.instruction_ptr]
        self.instruction_ptr += 1
        
        if mode == self.PARAM_MODE_POS:
            self.memory[param] = data
        else:
            raise Exception("Set paramater can't be in instant mode")
        
    def parse_opcode(self):
        mode_opcode_str = '{:>05}'.format(str(self.memory[self.instruction_ptr]))
        
        # Reverse of the first three chars
        modes = mode_opcode_str[:3][::-1]
        
        # integer of the last two chars
        opcode = int(mode_opcode_str[3:])
                     
        self.instruction_ptr += 1
                     
        return modes, opcode
        

    def run(self):
        self.output = None
        
        while self.instruction_ptr < len(self.memory):
            mode, opcode = self.parse_opcode()
            
            if opcode == self.OP_HALT:
                break
                
            elif opcode == self.OP_ADD:
                in1 = self.get_paramater(mode[0])
                in2 = self.get_paramater(mode[1])
                self.set_paramater(mode[2], in1 + in2)
            
            elif opcode == self.OP_MULTIPLY:
                in1 = self.get_paramater(mode[0])
                in2 = self.get_paramater(mode[1])
                self.set_paramater(mode[2], in1 * in2)
            
            elif opcode == self.OP_INPUT:
                if self.inputs:
                    self.last_input = self.inputs.pop()
                    
                if self.last_input != None:
                    self.set_paramater(mode[0], self.last_input)
                else:
                    raise Exception(f"{self.last_input} is not a valid input")
                
            elif opcode == self.OP_OUTPUT:
                self.output = self.get_paramater(mode[0])
                break # It took me way to long to realize I need to break here
                
            elif opcode == self.OP_JUMP_TRUE:
                do_jump = self.get_paramater(mode[0])
                new_addr = self.get_paramater(mode[1])
                if do_jump != 0:
                    self.instruction_ptr = new_addr
                
            elif opcode == self.OP_JUMP_FALSE:
                do_jump = self.get_paramater(mode[0])
                new_addr = self.get_paramater(mode[1])
                if do_jump == 0:
                    self.instruction_ptr = new_addr
                    
            elif opcode == self.OP_LESS_THAN:
                in1 = self.get_paramater(mode[0])
                in2 = self.get_paramater(mode[1])
                if in1 < in2:
                    self.set_paramater(mode[2], 1)
                else:
                    self.set_paramater(mode[2], 0)
                
            elif opcode == self.OP_EQUALS:
                in1 = self.get_paramater(mode[0])
                in2 = self.get_paramater(mode[1])
                if in1 == in2:
                    self.set_paramater(mode[2], 1)
                else:
                    self.set_paramater(mode[2], 0)
                
            else:
                raise Exception(f'Unknown opcode {opcode} at addr {self.instruction_ptr}.')
                self.reset()
                
        return self.output
    
def calc_max_thrust(intcode, phase_settings):
    last_output = 0
    iccp = IntcodeComputer(intcode)
    for phase in phase_settings:
        iccp.add_input(last_output)
        iccp.add_input(phase)
        iccp.run()
        if iccp.output != None:
            last_output = iccp.output
        iccp.reset()
    return last_output

def calc_max_thrust_feedback_loop(intcode, phase_settings, verbose = False):
    amp_index = 0
    amps = []
    for phase in phase_settings:
        amps.append(IntcodeComputer(intcode))
    amp_names = ['ampA','ampB','ampC','ampD','ampE']
    last_output = 0
    
    # Reverse so we can pop values
    phase_settings = phase_settings[::-1]
    
    while True:
        if verbose:
            amp_name = amp_names[amp_index % len(amp_names)]
            print(f'Running {amp_name} with input {last_output}')
    
        iccp = amps[amp_index % len(amps)]
        amp_index += 1

        iccp.add_input(last_output)
        
        if phase_settings:
            iccp.add_input(phase_settings.pop())
        
        iccp.run()
        if iccp.output != None:
            last_output = iccp.output
        else:
            break
            
    return last_output

class TestBasic(unittest.TestCase):
        
    def test_max_thrust(self):
        data = [([3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0], [4,3,2,1,0], 43210),
                ([3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0], [0,1,2,3,4], 54321),
                ([3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0], [1,0,4,3,2], 65210)]
        for intcode, phase_settings, target_thrust in data:
            self.assertEqual( calc_max_thrust(intcode, phase_settings), target_thrust )
        
    def test_max_thrust_loop(self):
        data = [([3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5], [9,8,7,6,5], 139629729),
                ([3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,-5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10], [9,7,8,5,6], 18216)]
        for intcode, phase_settings, target_thrust in data:
            self.assertEqual( calc_max_thrust_feedback_loop(intcode, phase_settings), target_thrust )
        
unittest.main(argv=[""], exit=False)


..
----------------------------------------------------------------------
Ran 2 tests in 0.006s

OK


In [2]:
from itertools import permutations

with open("inputs/input_d07.txt") as file:
    intcode = file.read()

all_possible_phase_settings = permutations(range(0,5))

val_to_thruster_list = []
for phase_settings in all_possible_phase_settings:
    val_to_thruster_list.append(calc_max_thrust(intcode, phase_settings))
    
max(val_to_thruster_list)

118936

<h2 id="part2">--- Part Two ---</h2>
<p>It's no good - in this configuration, the amplifiers can't generate a large enough output signal to produce the thrust you'll need. The Elves quickly talk you through rewiring the amplifiers into a <em>feedback loop</em>:</p>
<pre><code>      O-------O  O-------O  O-------O  O-------O  O-------O
0 -+-&gt;| Amp A |-&gt;| Amp B |-&gt;| Amp C |-&gt;| Amp D |-&gt;| Amp E |-.
   |  O-------O  O-------O  O-------O  O-------O  O-------O |
   |                                                        |
   '--------------------------------------------------------+
                                                            |
                                                            v
                                                     (to thrusters)
</code></pre>
<p>Most of the amplifiers are connected as they were before; amplifier <code>A</code>'s output is connected to amplifier <code>B</code>'s input, and so on. <em>However,</em> the output from amplifier <code>E</code> is now connected into amplifier <code>A</code>'s
    input. This creates the feedback loop: the signal will be sent through the amplifiers <em>many times</em>.</p>
<p>In feedback loop mode, the amplifiers need <em>totally different phase settings</em>: integers from <code>5</code> to <code>9</code>, again each used exactly once. These settings will cause the Amplifier Controller Software to repeatedly take input and
    produce output many times before halting. Provide each amplifier its phase setting at its first input instruction; all further input/output instructions are for signals.</p>
<p>Don't restart the Amplifier Controller Software on any amplifier during this process. Each one should continue receiving and sending signals until it halts.</p>
<p>All signals sent or received in this process will be between pairs of amplifiers except the very first signal and the very last signal. To start the process, a <code>0</code> signal is sent to amplifier <code>A</code>'s input <em>exactly once</em>.</p>
<p>Eventually, the software on the amplifiers will halt after they have processed the final loop. When this happens, the last output signal from amplifier <code>E</code> is sent to the thrusters. Your job is to <em>find the largest output signal that can be sent to the thrusters</em>    using the new phase settings and feedback loop arrangement.</p>
<p>Here are some example programs:</p>
<ul>
    <li>
        <p>Max thruster signal <em><code>139629729</code></em> (from phase setting sequence <code>9,8,7,6,5</code>):</p>
        <pre><code>3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,<br>27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5</code></pre>
    </li>
    <li>
        <p>Max thruster signal <em><code>18216</code></em> (from phase setting sequence <code>9,7,8,5,6</code>):</p>
        <pre><code>3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,<br>-5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,<br>53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10</code></pre>
    </li>
</ul>
<p>Try every combination of the new phase settings on the amplifier feedback loop. <em>What is the highest signal that can be sent to the thrusters?</em></p>

In [3]:
from itertools import permutations

with open("inputs/input_d07.txt") as file:
    intcode = file.read()

all_possible_phase_settings = permutations(range(5,10))

val_to_thruster_list = []
for phase_settings in all_possible_phase_settings:
    val_to_thruster_list.append(calc_max_thrust_feedback_loop(intcode, list(phase_settings)))
    
max(val_to_thruster_list)

57660948